# Título  Feature Engineering
## Autor: Jose Chelquer
## Fecha de última modificación: 22/11/2024
## Descripción:

Agrega features:
Sumas de variables (con o sin eliminación de las originales)
decilado de todas las variables de dinero (con o sin eliminación de las originales): eliminado por ahora porque agrega demasiado y no parece ser muy útil
lag1, 2 de todas salvo las obvias que no: numero_de_cliente, foto_mes, clase_ternaria
avg y ratios de las variables de dinero



## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [103]:
experimento="sps_"  # partiendo de la base
tipo_registros=experimento[0]            # '_'completo s sampleado
con_preprocesamiento = experimento[1]    # p con preprocesamiento  '_' sin preprocesamiento
tipo_featuring=experimento[2]            # _: 155 base  s: simple    c: completo
rf=experimento[3]                        # _: sin RF   r: con RF

dict_tipo_registros={'_': 'completo', 's': 'sampleado'}
dict_con_preprocesamiento={'_': 'sin preprocesamiento', 'p':'con preprocesamiento'}
dict_tipo_featuring={'_': 'base', 's':' simple', 'c': 'completo'}
dict_rf={'_': 'sin RF', 'f':' con RF'}

print (f'Experimento {experimento}')
print (dict_tipo_registros[tipo_registros])
print (dict_con_preprocesamiento[con_preprocesamiento])
print (dict_tipo_featuring[tipo_featuring])
print (dict_rf[rf])

Experimento sps_
sampleado
con preprocesamiento
 simple
sin RF


In [104]:
vm=True
if vm:
  usar_gdrive=False
else:
  usar_gdrive=True


In [73]:
if tipo_featuring=="s": #featuring simple

    hacer_sumas=True
    hacer_decilado=True
    a_decilar_eliminar_originales=False     # Eliminar las variables originales después de decilarlas
    a_sumar_eliminar_originales=False        # Eliminar las variables sumadas después de crear las sumas
    hacer_avg=False
    hacer_ratios=False
    eliminar_avg=True        # después de crear los ratios
    hacer_lags=True
if tipo_featuring == "c":      # completo
    hacer_sumas=True
    hacer_decilado=True
    a_decilar_eliminar_originales=False     # Eliminar las variables originales después de decilarlas
    a_sumar_eliminar_originales=False        # Eliminar las variables sumadas después de crear las sumas
    hacer_avg=True
    hacer_ratios=True
    eliminar_avg=False        #después de crear los ratios
    hacer_lags=True

In [74]:
# A decilar
#         Va a decilar las columnas de dinero
a_decilar_eliminar_originales=False     # Eliminar las variables originales después de decilarlas
# Columnas a sumar
a_sumar_eliminar_originales=False        # Eliminar las variables sumadas después de crear las sumas
a_sumar = [
    [['ctarjeta_visa', 'ctarjeta_master'], 'ctarjeta'],
    [['ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones'], 'ctarjeta_transacciones'],
    [['mtarjeta_visa_consumo', 'mtarjeta_master_consumo'], 'mtarjeta_consumo'],
    [['cplazo_fijo', 'cinversion1', 'cinversion2'], 'cinversion'],
    [['mplazo_fijo_pesos', 'minversion1_pesos', 'minversion2'], 'minversion_pesos'],
    [['mplazo_fijo_dolares', 'minversion1_dolares'], 'minversion_dolares'],
    [['cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales'], 'cseguro'],
    [['ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'], 'ctarjeta_debitos_automaticos'],
    [['mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'], 'mtarjeta_debitos_automaticos'],
    [['cpagodeservicios', 'cpagomiscuentas'], 'cpagos'],
    [['mpagodeservicios', 'mpagomiscuentas'], 'mpagos'],
    [['Master_msaldopesos', 'Visa_msaldopesos'], 'Tarjeta_msaldopesos'],
    [['Master_msaldodolares', 'Visa_msaldodolares'], 'Tarjeta_msaldodolares'],
    [['Master_msaldototal', 'Visa_msaldototal'], 'Tarjeta_msaldototal'],
    [['Master_mconsumospesos', 'Visa_mconsumospesos'], 'Tarjeta_mconsumospesos'],
    [['Master_mconsumosdolares', 'Visa_mconsumosdolares'], 'Tarjeta_mconsumosdolares'],
    [['Master_mlimitecompra', 'Visa_mlimitecompra'], 'Tarjeta_mlimitecompra'],
    [['Master_madelantopesos', 'Visa_madelantopesos'], 'Tarjeta_madelantopesos'],
    [['Master_madelantodolares', 'Visa_madelantodolares'], 'Tarjeta_madelantodolares'],
    [['Master_mpagado', 'Visa_mpagado'], 'Tarjeta_mpagado'],
    [['Master_mpagospesos', 'Visa_mpagospesos'], 'Tarjeta_mpagospesos'],
    [['Master_mpagosdolares', 'Visa_mpagosdolares'], 'Tarjeta_mpagosdolares'],
    [['Master_mconsumototal', 'Visa_mconsumototal'], 'Tarjeta_mconsumototal'],
    [['Master_cadelantosefectivo', 'Visa_cadelantosefectivo'], 'Tarjeta_cadelantosefectivo'],
    [['Master_mpagominimo', 'Visa_mpagominimo'], 'Tarjeta_mpagominimo'],

]


## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [75]:
if vm:
  dataset_path = '~/buckets/b1/datasets'
  dataset_file = f'k2_{tipo_registros}{con_preprocesamiento}_{rf}.parquet.gz'
else:
  dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
  dataset_file='k2_pre.parquet.gz'

print (dataset_path)
print (dataset_file)

~/buckets/b1/datasets
k2_sp.parquet.gz


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [76]:
output_file=f'k2_{tipo_registros}{con_preprocesamiento}{tipo_featuring}{rf}.parquet.gz'
print (output_file)
if vm:
  db_path='~/buckets/b1/db'
  db_file='competencia_02b.db'
else:
  db_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/db'
  db_file='competencia_02b.db'



k2_sps.parquet.gz


## Procesos

### Paquetes necesarios

In [77]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [78]:
if usar_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

In [79]:
import duckdb
import pandas as pd
import os



In [80]:
input_gz=dataset_file.endswith('.gz')
output_gz=output_file.endswith('.gz')

In [81]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
db_path_file=os.path.join(db_path, db_file)

conn = duckdb.connect(db_path_file)

In [82]:
def do_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  print(resultado)

def get_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  return resultado

# Obtiene lista de columnas
def columnas(tabla):
  sql=f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = '{tabla}';
  """
  lista=get_sql(sql)
  lista=[elemento[0] for elemento in lista]
  return lista

def registros(tabla):
     sql=f"""
      SELECT COUNT(*)
      FROM {tabla};
      """
     recs=get_sql(sql)
     return recs
def df_tabla(tabla):
    df = conn.execute(f"select * from {tabla}").fetchdf()
    return df
    
def cli8():
    df = conn.execute("SELECT * FROM competencia_02 WHERE foto_mes = 202108").fetchdf()
    cli8=df[df['foto_mes']==202108]
    registros=cli8.shape[0]
    print (registros)
    print (df['numero_de_cliente'].head(10))
    return (df)
    
    


## Leer datos

In [83]:
file_path

'~/buckets/b1/datasets/k2_sp.parquet.gz'

In [84]:
# Leer el parquet y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
sql = f"CREATE TABLE competencia_02 AS SELECT * FROM read_parquet('{file_path}')"
print (sql)
conn.execute(sql)

print (f'Cant de columnas leídas:{len(columnas("competencia_02"))}')


CREATE TABLE competencia_02 AS SELECT * FROM read_parquet('~/buckets/b1/datasets/k2_sp.parquet.gz')

  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
Cant de columnas leídas:155


In [85]:
cli8()

165442
0    249221109
1    249221468
2    249223005
3    249228180
4    249232117
5    249236712
6    249236857
7    249237079
8    249237446
9    249239632
Name: numero_de_cliente, dtype: int64


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249221109,202108,1,0,0,62,307,146.12,16161.50,3022.92,...,6.0,0.0,-30132.14,0.0,8079.0,92045.43,13.0,0.0,5665.59,BAJA+1
1,249221468,202108,1,0,0,53,44,9985.32,36911.77,5829.64,...,6.0,0.0,-96933.20,0.0,1336.0,70790.81,26.0,0.0,0.00,BAJA+1
2,249223005,202108,1,0,0,49,209,4747.14,38080.07,1146.31,...,6.0,0.0,-39365.88,0.0,3698.0,3479.27,4.0,0.0,20093.49,BAJA+1
3,249228180,202108,1,0,0,66,327,1915.94,2691.40,2190.03,...,6.0,0.0,-25926.81,0.0,9137.0,23664.14,6.0,0.0,1782.96,BAJA+1
4,249232117,202108,1,0,0,80,380,876.62,35718.76,573.10,...,6.0,0.0,-55905.09,0.0,8819.0,48958.87,7.0,0.0,4363.56,BAJA+1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165437,1603590310,202108,0,0,0,58,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,5.0,NaN,NaN,NaN,0.00,BAJA+1
165438,1603703854,202108,0,0,0,45,1,2.00,2.00,0.00,...,6.0,0.0,NaN,NaN,14.0,NaN,NaN,NaN,0.00,BAJA+1
165439,1603775178,202108,1,0,0,33,1,103.92,103.92,28.11,...,6.0,0.0,NaN,NaN,7.0,NaN,NaN,NaN,0.00,BAJA+1
165440,1603805076,202108,0,0,0,36,1,0.00,0.00,0.00,...,13.0,0.0,NaN,NaN,14.0,NaN,NaN,NaN,0.00,BAJA+1


## Listas de de variables

In [86]:
# Variables dinero originales (después se le agregarán las creadas en base a ellas)
dinero=[
 'mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheques_depositados_rechazados',
 'mcheques_emitidos_rechazados',
 'matm',
 'matm_other',
 'Master_mfinanciacion_limite',
 'Master_msaldototal',
 'Master_msaldopesos',
 'Master_msaldodolares',
 'Master_mconsumospesos',
 'Master_mconsumosdolares',
 'Master_mlimitecompra',
 'Master_madelantopesos',
 'Master_madelantodolares',
 'Master_mpagado',
 'Master_mpagospesos',
 'Master_mpagosdolares',
 'Master_mconsumototal',
 'Master_mpagominimo',
 'Visa_mfinanciacion_limite',
 'Visa_msaldototal',
 'Visa_msaldopesos',
 'Visa_msaldodolares',
 'Visa_mconsumospesos',
 'Visa_mconsumosdolares',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_madelantodolares',
 'Visa_mpagado',
 'Visa_mpagospesos',
 'Visa_mpagosdolares',
 'Visa_mconsumototal',
 'Visa_mpagominimo',
]





In [87]:
# Crea macro de suma_sin_null
do_sql('CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);')


CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);
[]


In [88]:
# Prepara las sumas sin null
def agregables_a(a_sumar, lista):
  # Seleccionan de todos los campo#
  # aquellos cuyos sumandos están en la lista
  a_agregar=[]
  for caso in a_sumar:
    if set(caso[0]).issubset(set(lista)):
      a_agregar.append(caso[1])
  return a_agregar


def crear_suma_sin_null_un_caso(caso):
  sumandos=caso[0]
  resultado=caso[1]
# Construye la línea
  linea=sumandos[0]

  sumandos=caso[0]
  suma=caso[1]
  linea=f'{sumandos[0]}'
  for sumando in sumandos[1:]:
    linea=f'suma_sin_null({sumando}, {linea})'
  linea=f'{linea} as {resultado}'
  return linea

def lista_suma_sin_null (a_sumar):
  lineas=[]
  for caso in a_sumar:
    lineas.append(crear_suma_sin_null_un_caso(caso))
  return lineas

def sql_sumar_sin_null(a_sumar, tabla):
  lista=lista_suma_sin_null(a_sumar)
  texto_con_saltos = "\n,".join(lista)
  sql=f'create or replace table {tabla} as \n  select *, {texto_con_saltos}\nfrom {tabla};'
  return sql

def lista_de_sumandos(a_sumar):
  return[elemento for sublista in a_sumar for elemento in sublista[0]]

def sql_eliminar_columnas(columnas, tabla):
    # Unir las columnas en un formato adecuado para el SQL
    sql = '; \n'.join([f"ALTER TABLE {tabla} DROP COLUMN {col}" for col in columnas])
    return sql

## Campos suma

In [89]:
if hacer_sumas:
    # Agrega los nuevos campos dinero
    dinero.extend(agregables_a(a_sumar, dinero))
    
    # Crea las variables suma
    sql=sql_sumar_sin_null(a_sumar, 'competencia_02')
    do_sql(sql)
    
    # Elimina las originales
    if a_sumar_eliminar_originales:
      sql=sql_eliminar_columnas(lista_de_sumandos(a_sumar), 'competencia_02')
      do_sql(sql)
    
    print (f'Cant de columnas:{len(columnas("competencia_02"))}')
else:
    print ("No se agregan campmos suma")

create or replace table competencia_02 as 
  select *, suma_sin_null(ctarjeta_master, ctarjeta_visa) as ctarjeta
,suma_sin_null(ctarjeta_master_transacciones, ctarjeta_visa_transacciones) as ctarjeta_transacciones
,suma_sin_null(mtarjeta_master_consumo, mtarjeta_visa_consumo) as mtarjeta_consumo
,suma_sin_null(cinversion2, suma_sin_null(cinversion1, cplazo_fijo)) as cinversion
,suma_sin_null(minversion2, suma_sin_null(minversion1_pesos, mplazo_fijo_pesos)) as minversion_pesos
,suma_sin_null(minversion1_dolares, mplazo_fijo_dolares) as minversion_dolares
,suma_sin_null(cseguro_accidentes_personales, suma_sin_null(cseguro_vivienda, suma_sin_null(cseguro_auto, cseguro_vida))) as cseguro
,suma_sin_null(ctarjeta_master_debitos_automaticos, ctarjeta_visa_debitos_automaticos) as ctarjeta_debitos_automaticos
,suma_sin_null(mttarjeta_master_debitos_automaticos, mttarjeta_visa_debitos_automaticos) as mtarjeta_debitos_automaticos
,suma_sin_null(cpagomiscuentas, cpagodeservicios) as cpagos
,suma_s

In [90]:
sql="""
CREATE OR REPLACE MACRO division_segura (a, b) AS (
  CASE
    WHEN b IS NULL THEN 0
    WHEN a IS NULL THEN 0
    WHEN b = 0 THEN 0
    WHEN a = 0 THEN 0
    ELSE a / b
  END
  )
"""
do_sql(sql)



CREATE OR REPLACE MACRO division_segura (a, b) AS (
  CASE
    WHEN b IS NULL THEN 0
    WHEN a IS NULL THEN 0
    WHEN b = 0 THEN 0
    WHEN a = 0 THEN 0
    ELSE a / b
  END
  )

[]


## Promedios y ratios

In [91]:
if hacer_avg:
    #Creo los promedios
    sql="""create or replace table promedios  as 
      select numero_de_cliente as pndc"""
    for campo in dinero:
        sql=sql+f',avg({campo}) as avg_{campo}\n'
    sql=sql+f"""
    from competencia_02 group by numero_de_cliente
    """
    do_sql(sql)
    
    print (f'Cant de columnas de promedios:{len(columnas("promedios"))}')
    # Agrego todos los campos de promedios a competencias
    sql="""
    create or replace table competencia_02 as
    SELECT
        c.*,  -- Todos los campos de competencia_02
        p.*   -- Todos los campos de promedios
    FROM
        competencia_02 c
    LEFT JOIN
        promedios p
    ON
        c.numero_de_cliente = p.pndc;
    """
    do_sql(sql)

    # Elimino el campo pndc
    sql="ALTER TABLE competencia_02 DROP COLUMN pndc;"
    do_sql(sql)
    
    print (f'Cant de columnas:{len(columnas("competencia_02"))}')
else:
    print ("NO se hacen promedios")

NO se hacen promedios


In [92]:
# Reordenar
sql="""
create or replace table competencia_02 as
SELECT * FROM competencia_02
ORDER BY foto_mes, numero_de_cliente;
"""
do_sql(sql)


create or replace table competencia_02 as
SELECT * FROM competencia_02
ORDER BY foto_mes, numero_de_cliente;

[(375309,)]


In [93]:
if hacer_ratios:
    # Ahora agrego los ratios
    sql="""create or replace table competencia_02 as
      select *"""
    for campo in dinero:
        sql=sql+f',division_segura({campo}, avg_{campo}) as ratio_{campo}\n'
    sql=sql+f"""
    from competencia_02
    """
    do_sql(sql)
    
    print (f'Cant de columnas:{len(columnas("competencia_02"))}')
else:
    print ("No se hacen ratios")

No se hacen ratios


In [94]:
if hacer_avg and eliminar_avg:
    # Elimino todas las columnas avg
    # Obtener las columnas que cumplen el patrón
    query = """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'competencia_02'
      AND column_name LIKE 'avg_%';
    """
    columns = conn.execute(query).fetchall()
    
    # Generar y ejecutar los comandos de eliminación
    sql=""
    for column in columns:
        sql=sql+f"ALTER TABLE competencia_02 DROP COLUMN {column[0]};\n"
    do_sql(sql)


In [95]:
# grabar competencia aumentada con features
def grabar_aumentada():
    output_file_path=os.path.join(dataset_path, output_file)
    if output_gz:
        print('Graba con gzip')
        sql=(f"COPY competencia_02 TO '{output_file_path}' (FORMAT PARQUET, COMPRESSION 'GZIP')")
    else:
        sql=(f"COPY competencia_02 TO '{output_file_path}' (FORMAT PARQUET)")
        print ('Graba sin gzip')
    do_sql(sql)
    print (f"Se grabó {output_file_path}")



## Decilar


In [96]:
if hacer_decilado:
    # Lo anulé: demora mucho, agrega mucho y no parece que sirva de mucho
    decilar=dinero
    #decilar=[]
    columnas_tabla=columnas('competencia_02')
    decilar = [elemento for elemento in decilar if elemento in columnas_tabla]
    orden=""
    orden=orden+"create or replace TABLE competencia_02 as\n"
    orden+=f"select *\n"
    if decilar:
        for campo in decilar:
          orden+=f",ntile(10) over (partition by foto_mes order by {campo}) as {campo}_n10\n"
        orden+=f"from competencia_02\n"
        
        
        do_sql(orden)
    
    print (f'Cant de columnas:{len(columnas("competencia_02"))}')

    if a_decilar_eliminar_originales:
      sql=sql_eliminar_columnas(decilar, 'competencia_02')
      do_sql(sql)


  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
create or replace TABLE competencia_02 as
select *
,ntile(10) over (partition by foto_mes order by mrentabilidad) as mrentabilidad_n10
,ntile(10) over (partition by foto_mes order by mrentabilidad_annual) as mrentabilidad_annual_n10
,ntile(10) over (partition by foto_mes order by mcomisiones) as mcomisiones_n10
,ntile(10) over (partition by foto_mes order by mactivos_margen) as mactivos_margen_n10
,ntile(10) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_n10
,ntile(10) over (partition by foto_mes order by mcuenta_corriente_adicional) as mcuenta_corriente_adicional_n10
,ntile(10) over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_n10
,ntile(10) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_n10
,ntile(10) over (partition by foto_mes order by mcaja_ahorro_adicional) as mcaja_ahorro_adicional_n10
,ntile(10) over (partitio

## Lags

In [97]:
if hacer_lags:
    ignorar=['numero_de_cliente', 'foto_mes', 'clase_ternaria']
    columnas_lag=columnas('competencia_02')
    a_lag=[elemento for elemento in columnas_lag if elemento not in ignorar]
    orden=""
    orden=orden+"create or replace TABLE competencia_02 as\n"
    orden+=f"select *\n"
    for campo in a_lag:
      for lag in [1, 2]:
        orden+=f",lag({campo}, {lag}) over (partition by numero_de_cliente order by foto_mes) as {campo}_lag{lag}\n"
    orden+=f"from competencia_02\n"
    
    do_sql(orden)
    print (f'Cant de columnas:{len(columnas("competencia_02"))}')



  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
create or replace TABLE competencia_02 as
select *
,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as active_quarter_lag1
,lag(active_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as active_quarter_lag2
,lag(cliente_vip, 1) over (partition by numero_de_cliente order by foto_mes) as cliente_vip_lag1
,lag(cliente_vip, 2) over (partition by numero_de_cliente order by foto_mes) as cliente_vip_lag2
,lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as internet_lag1
,lag(internet, 2) over (partition by numero_de_cliente order by foto_mes) as internet_lag2
,lag(cliente_edad, 1) over (partition by numero_de_cliente order by foto_mes) as cliente_edad_lag1
,lag(cliente_edad, 2) over (partition by numero_de_cliente order by foto_mes) as cliente_edad_lag2
,lag(cliente_antiguedad, 1) over (partition by numero_de_cliente order 

In [98]:
print (f'Cant de columnas:{len(columnas("competencia_02"))}')


  SELECT column_name
  FROM information_schema.columns
  WHERE table_name = 'competencia_02';
  
Cant de columnas:807


In [99]:
grabar_aumentada()

Graba con gzip
COPY competencia_02 TO '~/buckets/b1/datasets/k2_sps.parquet.gz' (FORMAT PARQUET, COMPRESSION 'GZIP')
[(375309,)]
Se grabó ~/buckets/b1/datasets/k2_sps.parquet.gz


In [100]:
conn.close()